In [1]:
import datetime as dt
import requests as req
import pandas as pd
import yfinance as yf
import sqlite3 as sqlite

In [2]:
def GET_INDICE_COMPONENTS():
    res  = req.get('https://www.slickcharts.com/sp500', headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' })
    page = pd.read_html(res.content)
    tbl  = page[0]
    return tbl

SNP_500 = GET_INDICE_COMPONENTS()['Symbol']

In [3]:
TICKERS = ['SPY','QQQ','IWM',*SNP_500.head(50)][:1]
TODAY   = dt.date.today()

In [4]:
def GET_AF_OPT_CHAINS(TICKERS):

    # ================ Helpers ================ #
    def _scrap(TICKER, EXPIRY, EXCHANGES=['NYSE','NASDAQ','AMEX'], STRIKES=50):
        for i, EXCHANGE in enumerate(EXCHANGES):
            try: 
                return pd.read_html(req.get(f"https://www.advfn.com/stock-market/{EXCHANGE}/{TICKER}/options?expiry={EXPIRY}&strikes={STRIKES}", headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' }).content)
            except: 
                if i+1 == len(EXCHANGES): 
                    return [pd.DataFrame(), pd.DataFrame()]
    

    # ================ Main ================ #
    Pipe = []
    for TICKER in TICKERS:
        Asset = yf.Ticker(TICKER)
        price = Asset.info.get('previousClose')

        for EXP in Asset.options:
            page = _scrap(TICKER, EXP.replace('-','')) 
            for OPT in [0,1]: 
                try: 
                    Opt = page[OPT]
                    Pipe.append(pd.DataFrame({
                        'Date':     TODAY, 
                        'Ticker':   TICKER, 
                        'Price':    price, 
                        'Opt':      { 0:'C', 1:'P' }[OPT], 
                        'Expiry':   EXP, 
                        'Strike':   Opt['Strike'], 
                        'Vol':      Opt['Volume'], 
                        'OI':       Opt['OPEN INT'], 
                        'Ask':      Opt['Ask Price'], 
                        'Mid':      Opt['Midpoint'], 
                        'Bid':      Opt['Bid Price'], 
                        'Last':     Opt['Last Price'], 
                    }))
                except Exception as E: print(E)
            pass
        pass
    pass
    return pd.concat(Pipe, ignore_index=1)

Chains = GET_AF_OPT_CHAINS(TICKERS)
Chains

,Date,Ticker,Price,Opt,Expiry,Strike,Vol,OI,Ask,Mid,Bid,Last
0,2024-11-05,SPY,571.04,C,2024-11-05,420.0,0,0,150.69,150.155,149.62,0.0
1,2024-11-05,SPY,571.04,C,2024-11-05,430.0,0,0,140.70,140.140,139.58,0.0
2,2024-11-05,SPY,571.04,C,2024-11-05,440.0,0,0,130.64,130.105,129.57,0.0
3,2024-11-05,SPY,571.04,C,2024-11-05,450.0,0,0,120.68,120.125,119.57,0.0
4,2024-11-05,SPY,571.04,C,2024-11-05,460.0,0,0,110.68,110.130,109.58,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6271,2024-11-05,SPY,571.04,P,2027-01-15,795.0,0,0,227.38,224.940,222.50,0.0
6272,2024-11-05,SPY,571.04,P,2027-01-15,800.0,0,0,232.38,229.940,227.50,0.0
6273,2024-11-05,SPY,571.04,P,2027-01-15,805.0,0,0,237.38,234.940,232.50,0.0
6274,2024-11-05,SPY,571.04,P,2027-01-15,810.0,0,0,242.38,239.940,237.50,0.0


In [ ]:
Chains.to_excel('Sample.xlsx')